In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from datasets import load_dataset

# Collect the training tasks

* Paraphrasing: https://huggingface.co/datasets/GEM/opusparcus
* Translation: https://huggingface.co/datasets/open_subtitles + news_commentary? + tatoeba?
* Detox: ordinary data

In [3]:
opus_para_en = load_dataset("GEM/opusparcus", "en.80")
opus_para_en

Reusing dataset opusparcus (/home/dale/.cache/huggingface/datasets/GEM___opusparcus/en.80/1.0.0/79d36ae4eced4f3c2c5a2ab9f94a584de7adca957186408d33798d0d87b018f2)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 982
    })
    validation: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1015
    })
    test.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1445
    })
    validation.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1455
    })
    train: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 5200000
    })
})

In [4]:
opus_para_ru = load_dataset("GEM/opusparcus", "ru.80")
opus_para_ru

Reusing dataset opusparcus (/home/dale/.cache/huggingface/datasets/GEM___opusparcus/ru.80/1.0.0/79d36ae4eced4f3c2c5a2ab9f94a584de7adca957186408d33798d0d87b018f2)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1068
    })
    validation: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1020
    })
    test.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1855
    })
    validation.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1854
    })
    train: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 2300000
    })
})

In [5]:
import random

In [6]:
random.choice(opus_para_ru['train'])

{'lang': 'ru',
 'input': 'Сделай, и всё.',
 'target': 'Просто сделай это.',
 'annot_score': 0.0,
 'gem_id': 'gem-opusparcus-train-78766378',
 'references': ['Просто сделай это.']}

In [7]:
opensub = load_dataset("open_subtitles", lang1="en", lang2='ru')
opensub

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset open_subtitles (/home/dale/.cache/huggingface/datasets/open_subtitles/en-ru-lang1=en,lang2=ru/0.0.0/c1ec973ca4b6e588740d8f167cc0e24ea3f626e70bc7ffe467e944730500e198)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'meta', 'translation'],
        num_rows: 25910105
    })
})

In [8]:
random.choice(opensub['train'])

{'id': '16790810',
 'meta': {'year': 2013,
  'imdbId': 2808666,
  'subtitleId': {'en': 5081422, 'ru': 5084906},
  'sentenceIds': {'en': [449], 'ru': [445]}},
 'translation': {'en': 'Tell me who she is.', 'ru': 'Скажи мне, кто она!'}}

In [9]:
news_commentary = load_dataset("news_commentary", lang1="en", lang2='ru')
news_commentary

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset news_commentary (/home/dale/.cache/huggingface/datasets/news_commentary/en-ru-lang1=en,lang2=ru/0.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 190104
    })
})

In [10]:
random.choice(news_commentary['train'])

{'id': '31817',
 'translation': {'en': 'Such measures aim to reduce the risk of conflict by increasing trust among participating OSCE states, and by contributing to greater transparency in the field of military planning and other activities.',
  'ru': 'Такие меры направлены на снижение риска возникновения конфликтов за счет увеличения доверия между государствами-участниками ОБСЕ, а также они будут способствовать повышению прозрачности в области военного планирования и других мероприятий.'}}

In [11]:
tatoeba = load_dataset("tatoeba", lang1="en", lang2='ru')
tatoeba

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset tatoeba (/home/dale/.cache/huggingface/datasets/tatoeba/en-ru-lang1=en,lang2=ru/0.0.0/b3ea9c6bb2af47699c5fc0a155643f5a0da287c7095ea14824ee0a8afd74daf6)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 523656
    })
})

In [12]:
random.choice(tatoeba['train'])

{'id': '404703',
 'translation': {'en': "Tom is from the same village that I'm from.",
  'ru': 'Мы с Томом из одной деревни.'}}

In [13]:
def get_paraphrase_task(batch_size=1):
    task = 'paraphrase: '
    if random.random() < 0.5:
        src = opus_para_en
        src_id, tgt_id = 'en_XX', 'en_XX'
    else:
        src = opus_para_ru
        src_id, tgt_id = 'ru_RU', 'ru_RU'
    x = []
    y = []
    for _ in range(batch_size):
        item = random.choice(src['train'])
        xx, yy = item['input'], item['target']
        if random.random() < 0.5:
            xx, yy = yy, xx
        x.append(task + xx)
        y.append(yy)
    return x, y, src_id, tgt_id
        
get_paraphrase_task(1)

(['paraphrase: Again, he wins.'], ['He won again.'], 'en_XX', 'en_XX')

In [14]:
def get_translate_task(batch_size=1):
    task = 'translate: '
    src = random.choice([tatoeba, opensub, news_commentary])
    src_id, tgt_id = 'ru_RU', 'en_XX'
    if random.random() < 0.5:
        src_id, tgt_id = tgt_id, src_id
    x = []
    y = []
    for _ in range(batch_size):
        item = random.choice(src['train'])
        x.append(task + item['translation'][src_id[:2]])
        y.append(item['translation'][tgt_id[:2]])
    return x, y, src_id, tgt_id

get_translate_task(2)

(['translate: Джон сказал: «Это красивая мелодия».',
  'translate: Не кидай камни в собаку.'],
 ['John said, "That\'s a beautiful tune."', "Don't throw stones at the dog."],
 'ru_RU',
 'en_XX')

In [15]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [16]:
model_name = '/home/dale/models/detox-parallel/mbart_5000_EN'
tokenizer = AutoTokenizer.from_pretrained('facebook/mbart-large-50')

In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda();

In [17]:
def get_batch(batch_size=4):
    x = []
    y = []
    bs = batch_size // 2
    for generator in [get_paraphrase_task, get_translate_task]:
        for _ in range(bs):
            xx, yy, src_id, tgt_id = generator(1)
            tokenizer.src_lang, tokenizer.tgt_lang = src_id, tgt_id
            x.append(tokenizer(xx))
            with tokenizer.as_target_tokenizer():
                y.append(tokenizer(yy))
    return x, y

In [41]:
x, y = get_batch(6)
x, y

([{'input_ids': [[250004, 121, 179665, 184, 12, 3293, 83, 10, 50208, 126, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250004, 121, 179665, 184, 12, 82310, 98676, 31766, 62, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250004, 121, 179665, 184, 12, 24080, 4, 3884, 442, 16065, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250004, 3900, 19309, 12, 992, 4398, 299, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250004, 3900, 19309, 12, 20, 10160, 4, 51544, 2355, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250021, 3900, 19309, 12, 56227, 26574, 77, 69, 33364, 41984, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}],
 [{'input_ids': [[250004, 442, 25, 7, 10, 50208, 126, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250004, 2583, 34614, 11, 1002, 775, 111, 10, 5, 2]]

In [42]:
for xx, yy in zip(x, y):
    print(tokenizer.decode(xx['input_ids'][0]))
    print(tokenizer.decode(yy['input_ids'][0]))

en_XX paraphrase: This is a bullet.</s>
en_XX it's a bullet.</s>
en_XX paraphrase: YOU SON OF A.</s>
en_XX You sneaky son of a.</s>
en_XX paraphrase: Nah, put it away.</s>
en_XX Never mind.</s>
en_XX translate: Uncanny.</s>
ru_RU Поразительно.</s>
en_XX translate: - Oh, thank god.</s>
ru_RU - Слава Богу!</s>
ru_RU translate: Нам никогда не дадут ребенка.</s>
en_XX We're never gonna get a kid.</s>


In [22]:
tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])

{'input_ids': [[250021, 121, 179665, 184, 12, 16434, 4042, 206677, 15856, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250004, 121, 179665, 184, 12, 175254, 23, 70, 131, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250004, 3900, 19309, 12, 62, 29954, 509, 7228, 23, 903, 48225, 23, 18237, 88922, 3229, 70, 66069, 111, 240596, 4, 172915, 25842, 1286, 3501, 23552, 111, 70, 32786, 117249, 111, 70, 14098, 145704, 4, 509, 75935, 23, 34210, 15519, 5, 2], [250004, 3900, 19309, 12, 6561, 199823, 43606, 75281, 136, 59671, 939, 9, 13, 22553, 47, 186, 24763, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

In [23]:
import transformers
transformers.logging.set_verbosity_error()

In [26]:
from transformers.optimization import Adafactor
optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=1e-5, clip_threshold=1.0)

In [27]:
from torch.optim.lr_scheduler import LambdaLR

num_warmup_steps = 1000

def lr_lambda(current_step: int):
    if current_step < num_warmup_steps:
        return float(current_step) / float(max(1.0, num_warmup_steps))
    return 1.0

scheduler = LambdaLR(optimizer, lr_lambda)

In [28]:
model.train()
model.gradient_checkpointing_enable()

In [29]:
from tqdm.auto import tqdm, trange

In [33]:
import torch
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [40]:
gradient_steps = 4
batch_size = 6
window = 1000
report_steps = 1000
cleanup_steps = 100
save_path = '/home/dale/models/detox-parallel/bart-multitask-pretrain'

In [36]:
ewm_loss = 4

In [37]:
cleanup()

With ~6 iterations per second, 1M iteration takes 1000000/6/60/60=45 hours.

In [ ]:
model.train()
tq = trange(160852 , 1_000_000)
cleanup()

for i in tq:
    x, y = get_batch(batch_size)
    xpad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])
    ypad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in y])
    
    try:
        labels = torch.tensor(ypad['input_ids'], device=model.device)
        labels[labels==tokenizer.pad_token_id] = -100
        loss = model(
            input_ids=torch.tensor(xpad['input_ids'], device=model.device),
            attention_mask=torch.tensor(xpad['attention_mask'], device=model.device),
            labels=labels,
        ).loss
        loss.backward()
        if i % gradient_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
    except RuntimeError as e:
        print('error', i, e)
        loss = None
        optimizer.zero_grad(set_to_none=True)
        cleanup()
        continue

    w = 1 / max(1, min(i, window))
    ewm_loss = ewm_loss * (1-w) + loss.item() * w
    tq.set_description(f'{ewm_loss:3.4f}')

    if i > 0 and i % report_steps == 0:
        print('step', i, 'loss', ewm_loss, )
        if i > 0:
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
    if i % cleanup_steps == 0:
        cleanup()

    

  0%|          | 0/839148 [00:00<?, ?it/s]

In [46]:
print(i, ewm_loss)
# step 141000 loss 0.9336458999525007
# 598943 0.7913156676625156
# 999999 0.7528545096685079

999999 1.076907632012897


```

step 1000 loss 4.030285649758136
step 2000 loss 2.924966513852863
step 4000 loss 2.0756006959416595
step 5000 loss 1.8793864612213047
step 6000 loss 1.7691763829453662
step 7000 loss 1.7182416955328477
step 8000 loss 1.6834452027635922
step 9000 loss 1.663543205332096
step 10000 loss 1.663113163950107
step 11000 loss 1.6201348936227629
step 12000 loss 1.6082521446914142
step 13000 loss 1.5896520319789016
step 14000 loss 1.5847235759135747
step 15000 loss 1.5505381041495554
step 16000 loss 1.543097093158245
step 17000 loss 1.5379276086863993
step 18000 loss 1.5277597218784424
step 19000 loss 1.5407952705743255
step 20000 loss 1.536612467038952
step 21000 loss 1.5191543397968388
step 22000 loss 1.518949046247585
step 23000 loss 1.503215299959398
step 24000 loss 1.5010942820045503
step 25000 loss 1.4981686735060622
step 26000 loss 1.4965668479089238
step 27000 loss 1.488605661614572
step 28000 loss 1.4758588817081453
step 29000 loss 1.4726483236694177
step 30000 loss 1.4814640238725008
step 31000 loss 1.47830836848471
step 32000 loss 1.4589154542656102
step 33000 loss 1.4417377788981833
step 34000 loss 1.4494910439710555
step 35000 loss 1.4426278926886749
step 36000 loss 1.44148649541007
step 37000 loss 1.4374428204636993
step 38000 loss 1.4578149832296017
step 39000 loss 1.4471952635777157
step 40000 loss 1.441363376735978
step 41000 loss 1.4358235478151893
step 42000 loss 1.4276501513380877
step 43000 loss 1.430725375910283
step 44000 loss 1.4313272358920095
step 45000 loss 1.432307047691588
step 46000 loss 1.4248935302674965
step 47000 loss 1.428975425487932
step 48000 loss 1.4367560202463607
step 49000 loss 1.4240793456122969
step 50000 loss 1.4081390656764776
step 51000 loss 1.4091096928337647
step 52000 loss 1.3977898006943437

step 53000 loss 1.408167125875017
step 54000 loss 1.411923371306938
step 55000 loss 1.394591920184885
step 56000 loss 1.4101064364472955
step 57000 loss 1.4045947166181658
step 58000 loss 1.4117634408999225
step 60000 loss 1.4005583499928356
step 61000 loss 1.3778290190471294
step 62000 loss 1.3916115126198214
step 63000 loss 1.3835347688222643
step 64000 loss 1.3984618459599827
step 65000 loss 1.3976124273679278
step 66000 loss 1.381784886532697
step 67000 loss 1.3648037205587196
step 68000 loss 1.374033853056438

step 69000 loss 1.3733464772640473
step 70000 loss 1.368408598845629
step 71000 loss 1.36885491135711
step 72000 loss 1.365724270276775
step 73000 loss 1.3748838410436879
step 74000 loss 1.3669939964287745
step 75000 loss 1.368752935215937
step 76000 loss 1.3712254828303787
step 77000 loss 1.3707613699794057
step 78000 loss 1.3723687654880743
step 79000 loss 1.3704682869621896
step 80000 loss 1.3620819050889972
step 81000 loss 1.365885952894489
step 82000 loss 1.3521919751292009
step 83000 loss 1.3528008621156438
step 84000 loss 1.3573856373971744
step 85000 loss 1.357475243249886
step 86000 loss 1.355344262885632
step 87000 loss 1.3396657697283283
step 88000 loss 1.3530322961749577
step 89000 loss 1.351423998173461
step 90000 loss 1.345621896752212
step 91000 loss 1.3491330424397554
step 92000 loss 1.3495505573811666
step 93000 loss 1.3458939547429194
step 94000 loss 1.3620523511085512
step 95000 loss 1.3511449763697705
step 96000 loss 1.3331114196031584
step 97000 loss 1.3442420122214624
step 98000 loss 1.3533528126214864
step 99000 loss 1.3576928244196738
step 100000 loss 1.3535247897620117
step 101000 loss 1.3387256920724813
step 102000 loss 1.3452958104057957
step 103000 loss 1.3356340940417883
step 104000 loss 1.333309402278774
step 105000 loss 1.3258874055995518
step 106000 loss 1.3370539755310682
step 107000 loss 1.343880505123775
step 108000 loss 1.341739257448781
step 109000 loss 1.3395123455102544
step 110000 loss 1.3279594339818224
step 111000 loss 1.3226150264791396
step 112000 loss 1.3206542441713167
step 113000 loss 1.3236204271528507
step 114000 loss 1.3193116127056803
step 115000 loss 1.3152623717347338
step 116000 loss 1.3086302777171588
step 117000 loss 1.3310852806315456
step 118000 loss 1.3327323135754303
step 119000 loss 1.3145623184089887
step 120000 loss 1.3062492088541324
step 121000 loss 1.3141538967149964
step 122000 loss 1.3160020090388171
step 123000 loss 1.321558573313111
step 124000 loss 1.3127376859450441
step 125000 loss 1.3138053120861193
step 126000 loss 1.2949118680851988
step 127000 loss 1.3034073680619607
step 128000 loss 1.3183906576789546
step 129000 loss 1.3019162163215015
step 130000 loss 1.3010734672083777
step 131000 loss 1.3191547748294705
step 132000 loss 1.3149730288162342
step 133000 loss 1.3171935941087558
step 134000 loss 1.3153115811646312
step 135000 loss 1.3181873168703298
step 136000 loss 1.3133259883003798
step 137000 loss 1.32182021592838
step 138000 loss 1.3079991855624757
step 139000 loss 1.2964033815500906
step 140000 loss 1.303071548959342
step 141000 loss 1.3062257196002987
step 142000 loss 1.2999398341278006
step 143000 loss 1.2982098372683213
step 144000 loss 1.3052906077292914
step 145000 loss 1.3063556414884758
step 146000 loss 1.3070851643857095
step 147000 loss 1.2984685859014025
step 148000 loss 1.3053904894859323
step 149000 loss 1.304871654869325
step 150000 loss 1.3008903165969232
step 151000 loss 1.303887735036512
step 152000 loss 1.2935637518539662
step 153000 loss 1.3060935613096878
step 154000 loss 1.302060990732172
step 155000 loss 1.2962453502285138
step 156000 loss 1.2921562535555324
step 157000 loss 1.2871806450898
step 158000 loss 1.2995706042796882
step 159000 loss 1.3095689668868038
step 160000 loss 1.2946405708000173
...
step 999999 loss 1.076907632012897
```

In [47]:
model.eval();

In [48]:
tokenizer.src_lang = 'ru_RU'

In [49]:
tokenizer.src_lang = 'en_XX'

In [50]:
tokenizer('привет')

{'input_ids': [250004, 146038, 2], 'attention_mask': [1, 1, 1]}

In [51]:
tokenizer.convert_tokens_to_ids('en_XX')

250004

In [52]:
def paraphrase(
    text, model, tokenizer, 
    n=None, 
    max_length="auto", 
    beams=5,
    src_lang='en_XX',
    tgt_lang='en_XX',
    **kwargs
):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    texts = [text] if isinstance(text, str) else text
    tokenizer.tgt_lang
    inputs = tokenizer(texts, return_tensors="pt", padding=True)["input_ids"].to(
        model.device
    )

    if max_length == "auto":
        max_length = inputs.shape[1] + 10

    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=10.0,
        max_length=max_length,
        min_length=int(0.5 * max_length),
        num_beams=beams,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]

    if not n and isinstance(text, str):
        return texts[0]
    return texts[0]

In [53]:
print(paraphrase('paraphrase: I like to play with my nice dog.', model, tokenizer))

I like to play with my dog, you know.


In [54]:
print(paraphrase('translate: I like to play with my nice dog.', model, tokenizer, tgt_lang='ru_RU'))

Мне нравится играть со своей милой собакой.


In [55]:
print(paraphrase('translate: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='ru_RU'))

Ненавижу играть со своей собачкой.


In [56]:
print(paraphrase('translate: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU'))

I hate playing with my fuckin' dog, man.


In [57]:
print(paraphrase('paraphrase: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like playing with my dog.


# Transfer from English to Russian

In [61]:
import pandas as pd

In [62]:
detox_en2ru = pd.read_csv('detox_en2ru_yandex.tsv', sep='\t')

In [63]:
detox_en2ru_filtered = detox_en2ru[
    (detox_en2ru.edit_distance_ru >= detox_en2ru.edit_distance_en.quantile(0.01)) 
    & (detox_en2ru.edit_distance_ru <= detox_en2ru.edit_distance_en.quantile(0.99)) 
    & (detox_en2ru.edit_sim_ru >= detox_en2ru.edit_sim_en.quantile(0.01)) 
    & (detox_en2ru.edit_sim_ru <= detox_en2ru.edit_sim_en.quantile(0.99))
]

print(detox_en2ru.shape)
print(detox_en2ru_filtered.shape)

(19766, 15)
(18488, 15)


In [64]:
detox_en2ru_filtered.sample(3)

,idx,toxic_comment,neutral_comment,toxicity_score,dataset,toxic,confidence_toxic,is_match,confidence_is_match,toxic_ru,neutral_ru,edit_distance_en,edit_distance_ru,edit_sim_en,edit_sim_ru
16628,111246,where the fuck did that tidbit come from ?,Where did this obscenity come from?,0.996669,reddit,False,0.9999,True,0.9536,"откуда, черт возьми, взялся этот лакомый кусочек?",Откуда взялась эта непристойность?,20,33,0.523810,0.326531
13005,209188,""" but they give a shit if you say you drive a ...",""" but they care if you say you drive a mazda w...",0.963981,reddit,False,0.9990,True,0.9727,""" но им насрать , если ты говоришь, что водишь...","""но им не все равно, если ты говоришь, что вод...",11,11,0.872093,0.890000
13545,100151,should be fucking common sense .,Should be common sense,0.988590,reddit,False,0.9906,True,0.9565,должен быть гребаный здравый смысл .,Должен быть здравый смысл,11,12,0.656250,0.666667


In [65]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(detox_en2ru_filtered, random_state=1, test_size=500)

In [76]:
def get_detox_task_en(batch_size=1):
    task = 'detox: '
    sample = train.sample(batch_size)
    src_id, tgt_id = 'en_XX', 'en_XX'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_comment)
        y.append(row.neutral_comment)
    return x, y, src_id, tgt_id

get_detox_task_en(2)

(['detox: this , again , needs context no , this fucking shit does not need any context at all .',
  'detox: it always is funny to me when someone religious talks shit about another religions views'],
 ['this , again , needs context no , this  does not need any context at all .',
  'it always is funny to me when someone religious talks bad about another religions views.'],
 'en_XX',
 'en_XX')

In [69]:
def get_detox_task_ru(batch_size=1):
    task = 'detox: '
    sample = train.sample(batch_size)
    src_id, tgt_id = 'ru_RU', 'ru_RU'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_ru)
        y.append(row.neutral_ru)
    return x, y, src_id, tgt_id

get_detox_task_ru(2)

(['detox:  и все они были говноедами .',
  'detox: я никогда не знал, что они богаты, просто глупые туристы в кричащих рубашках алоха.'],
 ['все они были любителями поговорить',
  'Я никогда не знал, что они богаты, просто туристы в кричащих рубашках алоха.'],
 'ru_RU',
 'ru_RU')

In [77]:
def get_batch(batch_size=4):
    x = []
    y = []
    bs = batch_size // 4
    for generator in [get_detox_task_en, get_detox_task_ru, get_paraphrase_task, get_translate_task]:
        for _ in range(bs):
            xx, yy, src_id, tgt_id = generator(1)
            tokenizer.src_lang, tokenizer.tgt_lang = src_id, tgt_id
            x.append(tokenizer(xx))
            with tokenizer.as_target_tokenizer():
                y.append(tokenizer(yy))
    return x, y

In [78]:
!ls /home/dale/models/detox-parallel

bart-multitask-pretrain
bart-multitask-v1
bart-multitask-v2
mbart_5000_EN
mbart_5000_RU
translate-en2ru-filter75best-rut5
translate-en2ru-full_aug_bilingual-mbart
translate-en2ru-full_aug_bilingual-mbart-finetune
translate-en2ru-full_bilingual-mbart
translate-en2ru-full-mbart
translate-en2ru-full-rut5
translate-en2ru_yandex-full-mbart
translate-ru2en-full-mbart
translate-ru2en_yandex-full_bilingual-mbart


In [79]:
batch_size = 8
save_path = '/home/dale/models/detox-parallel/bart-multitask-v3-ru'

In [80]:
model.train()
tq = trange(0, 50_000)
cleanup()

for i in tq:
    x, y = get_batch(batch_size)
    xpad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])
    ypad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in y])
    
    try:
        labels = torch.tensor(ypad['input_ids'], device=model.device)
        labels[labels==tokenizer.pad_token_id] = -100
        loss = model(
            input_ids=torch.tensor(xpad['input_ids'], device=model.device),
            attention_mask=torch.tensor(xpad['attention_mask'], device=model.device),
            labels=labels,
        ).loss
        loss.backward()
        if i % gradient_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
    except RuntimeError as e:
        print('error', i, e)
        loss = None
        optimizer.zero_grad(set_to_none=True)
        cleanup()
        continue

    w = 1 / max(1, min(i, window))
    ewm_loss = ewm_loss * (1-w) + loss.item() * w
    tq.set_description(f'{ewm_loss:3.4f}')

    if i > 0 and i % report_steps == 0:
        print('step', i, 'loss', ewm_loss, )
        if i > 0:
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
    if i % cleanup_steps == 0:
        cleanup()

  0%|          | 0/50000 [00:00<?, ?it/s]

step 1000 loss 1.0244991126060508
error 1644 CUDA out of memory. Tried to allocate 1.07 GiB (GPU 0; 10.76 GiB total capacity; 7.93 GiB already allocated; 845.69 MiB free; 8.81 GiB reserved in total by PyTorch)
step 2000 loss 0.9926564970535389
step 3000 loss 0.9637185616676812
step 4000 loss 0.9397538064847123
step 5000 loss 0.9433629860885504
error 5234 CUDA out of memory. Tried to allocate 1.48 GiB (GPU 0; 10.76 GiB total capacity; 8.26 GiB already allocated; 701.69 MiB free; 8.95 GiB reserved in total by PyTorch)
error 5238 CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 10.76 GiB total capacity; 6.83 GiB already allocated; 953.69 MiB free; 8.70 GiB reserved in total by PyTorch)
error 5366 CUDA out of memory. Tried to allocate 848.00 MiB (GPU 0; 10.76 GiB total capacity; 7.74 GiB already allocated; 735.69 MiB free; 8.92 GiB reserved in total by PyTorch)
error 5484 CUDA out of memory. Tried to allocate 810.00 MiB (GPU 0; 10.76 GiB total capacity; 7.59 GiB already allocated; 

In [81]:
model.eval();

In [82]:
print(paraphrase('paraphrase: I like to play with my nice dog.', model, tokenizer))

I'd like to play with my dog.


In [83]:
print(paraphrase('translate: I like to play with my nice dog.', model, tokenizer, tgt_lang='ru_RU'))

Я люблю играть со своей милой собакой.


In [84]:
print(paraphrase('translate: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='ru_RU'))

Я ненавижу играть со своей собачкой.


In [85]:
print(paraphrase('translate: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU'))

I hate playing with my fucking dog. Oh, no!


In [86]:
print(paraphrase('paraphrase: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like to play with my dog.


In [87]:
print(paraphrase('detox: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like to play with my dog.


In [88]:
print(paraphrase('detox: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU'))

Ненавижу играть со своей собакой, черт возьми.


In [90]:
test_data = pd.read_csv('../data/russian_data/test.tsv', sep='\t')
test_inputs = test_data["toxic_comment"].values.tolist()

In [91]:
test_outputs = [paraphrase('detox: ' + text, model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU') for text in tqdm(test_inputs)]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [94]:
path = '../results/multitask-1M-translate-yandex-v3/'  # partial training results; need to train further.
if not os.path.exists(path):
    os.mkdir(path)

In [95]:
with open(path + 'results_ru.txt', 'w') as f:
    for line in test_outputs:
        f.write(line + '\n')

Full results (1m steps)  + 50K with Russian translated data
```
python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/multitask-1M-translate-yandex-v3 \
    --output_dir results
    
Style accuracy:       0.6365973949432373
Meaning preservation: 0.861542284488678
Joint fluency:        -0.09916463494300842
Joint score:          -0.052923914045095444
Scores after calibration:
Style accuracy:       0.6729376912117004
Meaning preservation: 0.792417585849762
Joint fluency:        0.8859606981277466
Joint score:          0.4621787667274475
```